In [ ]:
# import all required packages

import requests, re
import pandas as pd
import numpy as np
import json
import xlsxwriter
from time import sleep
import random
import datetime

In [ ]:
# functions to be used are defined here

def get_address(ticker):

    try:
        url = 'https://finance.yahoo.com/quote/{}/profile'.format(ticker)
        req = requests.get(url)
        txt = re.search(r'root\.App\.main = (.*?);\n}\(this\)\);', req.text, re.DOTALL).group(1)
        jsn = json.loads(txt)
        street = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['address1']
        city = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['city']
        try:
            state = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['state']
            zip_code = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['zip']
            address = (str.join(', ',[street,city,state,zip_code]))
        except:
            country = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['country']
            address = (str.join(', ',[street,city,country]))
    
        return address
    
    except:
        address = np.nan
    
    return address

def geolocate(ticker):

    try:
        address = get_address(ticker)
        API_KEY = 'bPuxRuQQPkHdDYLApq18OB3UesUo9r5O'
        url_2 = 'http://www.mapquestapi.com/geocoding/v1/address'
        params = {'key': API_KEY, 'location': address}
        req = requests.get(url_2, params=params)
        lat = float(re.search(r'\"latLng":{"lat":([?-]?\d*\.\d*),"lng":([?-]?\d*\.\d*)}', req.text).group(1))
        lng = float(re.search(r'\"latLng":{"lat":([?-]?\d*\.\d*),"lng":([?-]?\d*\.\d*)}', req.text).group(2))
        #geo_location = tuple([lat,lng,address])
        return lat, lng
    
    except:
        lat = np.nan
        lng = np.nan
        return lat, lng
    
def get_data(ticker):

# Using the statistics section from Yahoo Finance

    url = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
    req = requests.get(url)
    txt = re.search(r'root\.App\.main = (.*?);\n}\(this\)\);', req.text, re.DOTALL).group(1)
    jsn = json.loads(txt)

    # Revenue
    revenue = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['financialData']['totalRevenue']['raw']

    # Cash Flow Margin
    operating_cash_flow = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['financialData']['operatingCashflow']['raw']
    cash_flow_margin = operating_cash_flow / revenue
   
    # Profitability (NI/Revenue)
   
    profit_margin = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['financialData']['profitMargins']['raw']
    
    # Growth

    earnings_growth = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['financialData']['earningsGrowth']['raw']
    
# Using the profile section from Yahoo Finnace

    url = f'https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}'
    req = requests.get(url)
    txt = re.search(r'root\.App\.main = (.*?);\n}\(this\)\);', req.text, re.DOTALL).group(1)
    jsn = json.loads(txt)
    
    # Sector
    try:
        sector = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['sector']
    except:
        sector = np.nan
    # Sub sector / Industry
    try:
        industry = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['industry']
    except:
        indsutry = np.nan
    # Address
    address = get_address(ticker)
    # Name
    try: 
        name = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['quoteType']['longName']
    except:
        name = np.nan

# Using the sustainability section from Yahoo Finance

    url = f'https://finance.yahoo.com/quote/{ticker}/sustainability?p={ticker}'
    req = requests.get(url)
    txt = re.search(r'root\.App\.main = (.*?);\n}\(this\)\);', req.text, re.DOTALL).group(1)
    jsn = json.loads(txt)
    
    # Overall ESG
    totalESG = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['totalEsg']['raw']
    # Percentile Rank
    totalPercentile = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['percentile']['raw']
    # Environment Score
    environmentScore = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['environmentScore']['raw']
    # Peer Environment
    peerEnvironment = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['peerEnvironmentPerformance']['avg']
    # Social Score
    socialScore = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['socialScore']['raw']
    # Peer Social
    peerSocial = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['peerSocialPerformance']['avg']
    # Governance Score
    governanceScore = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['governanceScore']['raw']
    # Peer Governance
    peerGovernance = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['peerGovernancePerformance']['avg']

    
# Using the balance sheet section from Yahoo Finance
    # Asset Turnover 

    url = f'https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}'
    req = requests.get(url)
    txt = re.search(r'root\.App\.main = (.*?);\n}\(this\)\);', req.text, re.DOTALL).group(1)
    jsn = json.loads(txt)
    asset_dict = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
    totalAssets = float(re.search(r'\'totalAssets\': {\'raw\': (\d*?),', str(asset_dict[0])).group(1))
    asset_turnover = revenue / totalAssets
    
# Using data from gurufocus

    url= f'https://www.gurufocus.com/term/quick_ratio/{ticker}/Quick-Ratio/'
    req = requests.get(url)
    html = req.text
    quick_ratio = float(re.search(r'has a Quick Ratio: (\d+.\d+)',html).group(1))

# Other
    # Average Annual Returns
    rets = pdr.get_data_yahoo(ticker)
    date_index = rets.index.to_pydatetime()
    date_timedelta = date_index.max() - date_index.min()
    date_timedelta = date_timedelta.days
    years = date_timedelta / 365
    first_price = rets['Adj Close'][0]
    last_price = rets['Adj Close'][-1]
    avg_annual_return = (last_price / first_price - 1) / years

# Location
    lat, lng = geolocate(ticker)

    return name, sector, industry, address, lat, lng, totalESG, totalPercentile, environmentScore, peerEnvironment, socialScore, peerSocial, governanceScore, peerGovernance, profit_margin, asset_turnover, quick_ratio, cash_flow_margin, avg_annual_return



In [ ]:
# import list of all stocks traded on the various exchanges

nasdaq = pd.read_csv('NASDAQ.csv')
nyse = pd.read_csv('NYSE.csv')
amex = pd.read_csv('AMEX.csv')
nasdaq = list(nasdaq.iloc[:,0])
nyse = list(nyse.iloc[:,0])
amex = list(amex.iloc[:,0])

# Create a master dataframe which will eventually contain all desired information
list_of_tickers = nasdaq + nyse + amex
dict_of_tickers = {'Ticker': list_of_tickers}
master_list = pd.DataFrame.from_dict(dict_of_tickers)
#master_list.drop_duplicates('Symbol')
master_list['Total ESG Score'] = np.nan
pd.set_option('display.max_rows', len(master_list))

# Searches Yahoo Finance to see if there is ESG information available

for i in range(len(master_list)):
    print(i)
    ticker = master_list.iloc[i,0]
    url = f'https://finance.yahoo.com/quote/{ticker}/sustainability'
    req = requests.get(url)
    html = req.text
    try:
        totalESG = jsn['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']['totalEsg']['raw']
        if totalESG:
            master_list.iloc[i,1] = totalESG
    except:
        pass
    
    sleep(0.1)

# Removes all rows that do not have an ESG percentile rank 

master_list = master_list.dropna()
master_list['Total ESG Score'] = pd.to_numeric(master_list['Total ESG Score'])
master_list = master_list.sort_index(ascending = True)
master_list = master_list.reset_index(drop = True)
master_list = master_list.set_index('Ticker')
master_list.to_excel('ESG_Data.xlsx')

In [ ]:
master_list = pd.read_excel('ESG_Data.xlsx')

In [ ]:
# Adds all desired information to the master dictionary
data = {}
for i in range(len(master_list)):
    print(i)
    ticker = master_list.iloc[i,0]
    print(ticker)
    try:
        data.update({ticker:get_data(ticker)})
    except:
        pass

In [ ]:
# format and save the master_df

master_df = pd.DataFrame(data).transpose()
master_df.index.rename('Ticker',inplace = True)
for i in range(4,len(master_df.columns)):
    master_df[i] = master_df[i].astype(float)
pd.set_option('display.float_format', '{:.4f}'.format)

columns = ['name','sector','industry', 'address','lat','lng', 'totalESG', 'totalPercentile', 
                     'environmentScore', 'peerEnvironment', 'socialScore', 'peerSocial', 'governanceScore',
                     'peerGovernance', 'profit_margin', 'asset_turnover','quick_ratio','cash_flow_margin', 
                     'avg_annual_return']
master_df.columns = columns
master_df.sort_index(inplace = True, ascending = True)
master_df.to_pickle('master_df.pkl')